In [1]:
import os

import pandas as pd

from tqdm.notebook import tqdm

from topic_extraction import get_relevant_nps_from_text, parse_text, get_tokens_with_stopwords

In [5]:
# base_dir = "/Users/ronpick/studies/stance/alternative/createdebate_released"
base_dir = "../data/fourforums"

In [6]:
cd_path = f"/Users/ronpick/workspace/zero-shot-stance/data/createdebate/iac-createdebate-subconvs.csv"
cddf = pd.read_csv(cd_path)
cddf

,discussion_id,post_id,author_id,creation_date,parent_post_id,parent_missing,text_id,points,discussion_stance_id,is_rebuttal,root_discussion_id,topic,title
0,878,3557,27083,2008-05-07 23:06:11,878.0,0,513135,-3,0.0,NaN,878,9,Should guns be banned in America?
1,878,3563,5901,2008-05-08 01:25:25,878.0,0,513160,-8,0.0,NaN,878,9,Should guns be banned in America?
2,878,3623,11010,2008-05-08 15:03:14,878.0,0,513211,11,1.0,NaN,878,9,Should guns be banned in America?
3,878,3627,27083,2008-05-08 15:21:42,3623.0,0,513212,8,0.0,disputed,878,9,Should guns be banned in America?
4,878,3632,8705,2008-05-08 15:35:54,878.0,0,513276,4,1.0,NaN,878,9,Should guns be banned in America?
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3046,56718,558884,1488,2014-05-09 05:30:18,56718.0,0,771300,1,0.0,NaN,56718,9,gun control
3047,56718,558914,2827,2014-05-09 13:40:52,558884.0,0,771301,1,1.0,disputed,56718,9,gun control
3048,56718,558915,2827,2014-05-09 13:41:09,56718.0,0,771304,1,1.0,NaN,56718,9,gun control
3049,56718,558927,2496,2014-05-09 16:26:02,558914.0,0,771302,1,1.0,supported,56718,9,gun control


In [7]:
texts_path = os.path.join(base_dir, "text.txt")
text_by_id = {}
with open(texts_path, 'r') as text_f:
    text_record = []
    for line in tqdm(text_f):
        line = line.strip()
        if len(line) >= 1 and line[-1] == '\\':
            text_record.append(line[:-2])
            continue

        if len(text_record) == 0:
            text_record.append(line.strip())

        try:
            text_id, first_text_part = text_record[0].split(maxsplit=1)
        except ValueError:
            text_id = text_record[0]
            first_text_part = ""

        text_record[0] = first_text_part
        text = "".join(text_record).strip()
        text_by_id[int(text_id.strip())] = text
        text_record = []

In [8]:
len(text_by_id)

3875

In [9]:
import json
records = []
for index, record in tqdm(cddf.iterrows()):
    text = text_by_id[record["text_id"]]
    parsed_doc = parse_text(text)
    tokens, pos = get_tokens_with_stopwords(parsed_doc)
    tokens_str = json.dumps(tokens)
    pos_str = json.dumps(pos)
    text_s = " ".join(" ".join(sent) + "." for sent in tokens)
    topics = get_relevant_nps_from_text(parsed_doc)
    for topic in topics:
        topic_str = " ".join(topic)
        vast_record = {
            "author": record["author_id"],
            "post": text,
            "ori_topic": topic_str,
            "ori_id": None,
            "new_topic": topic_str,
            "label": -1,
            "type_idx": 1,
            "new_id": f"cd-{record['post_id']}",
            "arc_id": None,
            "text": tokens_str,
            "pos_text": pos_str,
            "text_s": text_s,
            "topic": json.dumps(topic),
            "topic_str": topic_str,
            "seen?": 0,
            "contains_topic?": 1
        }
        records.append(vast_record)

cdvast_df = pd.DataFrame.from_records(records)
cdvast_df

,author,post,ori_topic,ori_id,new_topic,label,type_idx,new_id,arc_id,text,pos_text,text_s,topic,topic_str,seen,contains_topic?
0,27083,Guns should be banned because they are not nee...,second ammendment,None,second ammendment,-1,1,cd-3557,None,"[[""guns"", ""banned"", ""needed"", ""domestic"", ""iss...","[[""NNS"", ""VBN"", ""VBN"", ""JJ"", ""NN""], [""JJ"", ""NN...",guns banned needed domestic issue. second amme...,[second ammendment],second ammendment,0,1
1,27083,Guns should be banned because they are not nee...,british,None,british,-1,1,cd-3557,None,"[[""guns"", ""banned"", ""needed"", ""domestic"", ""iss...","[[""NNS"", ""VBN"", ""VBN"", ""JJ"", ""NN""], [""JJ"", ""NN...",guns banned needed domestic issue. second amme...,[british],british,0,1
2,27083,Guns should be banned because they are not nee...,age america,None,age america,-1,1,cd-3557,None,"[[""guns"", ""banned"", ""needed"", ""domestic"", ""iss...","[[""NNS"", ""VBN"", ""VBN"", ""JJ"", ""NN""], [""JJ"", ""NN...",guns banned needed domestic issue. second amme...,[age america],age america,0,1
3,27083,Guns should be banned because they are not nee...,close enemies,None,close enemies,-1,1,cd-3557,None,"[[""guns"", ""banned"", ""needed"", ""domestic"", ""iss...","[[""NNS"", ""VBN"", ""VBN"", ""JJ"", ""NN""], [""JJ"", ""NN...",guns banned needed domestic issue. second amme...,[close enemies],close enemies,0,1
4,27083,Guns should be banned because they are not nee...,easier crimes,None,easier crimes,-1,1,cd-3557,None,"[[""guns"", ""banned"", ""needed"", ""domestic"", ""iss...","[[""NNS"", ""VBN"", ""VBN"", ""JJ"", ""NN""], [""JJ"", ""NN...",guns banned needed domestic issue. second amme...,"[easier, crimes]",easier crimes,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8080,2827,Making firearms illegal only stops law abiding...,only difference,None,only difference,-1,1,cd-558915,None,"[[""making"", ""firearms"", ""illegal"", ""stops"", ""l...","[[""VBG"", ""NNS"", ""JJ"", ""VBZ"", ""NN"", ""VBG"", ""NNS...",making firearms illegal stops law abiding citi...,[only difference],only difference,0,1
8081,2827,Making firearms illegal only stops law abiding...,black market distribution profitsome drug users,None,black market distribution profitsome drug users,-1,1,cd-558915,None,"[[""making"", ""firearms"", ""illegal"", ""stops"", ""l...","[[""VBG"", ""NNS"", ""JJ"", ""VBZ"", ""NN"", ""VBG"", ""NNS...",making firearms illegal stops law abiding citi...,"[black market, distribution, profitsome drug u...",black market distribution profitsome drug users,0,1
8082,2827,Making firearms illegal only stops law abiding...,criminals,None,criminals,-1,1,cd-558915,None,"[[""making"", ""firearms"", ""illegal"", ""stops"", ""l...","[[""VBG"", ""NNS"", ""JJ"", ""VBZ"", ""NN"", ""VBG"", ""NNS...",making firearms illegal stops law abiding citi...,[criminals],criminals,0,1
8083,2496,"Yeah, I thought that was hilarious.",hilarious,None,hilarious,-1,1,cd-558927,None,"[[""yeah"", ""thought"", ""hilarious""]]","[[""UH"", ""VBD"", ""JJ""]]",yeah thought hilarious.,[hilarious],hilarious,0,1


In [14]:
cdvast_df.insert(14, "seen?", cdvast_df["seen"])
del cdvast_df["seen"]

In [16]:
cdvast_df.to_csv("/Users/ronpick/workspace/zero-shot-stance/data/createdebate/cdvast.csv")


In [15]:
cdvast_df

,author,post,ori_topic,ori_id,new_topic,label,type_idx,new_id,arc_id,text,pos_text,text_s,topic,topic_str,seen?,contains_topic?
0,27083,Guns should be banned because they are not nee...,second ammendment,None,second ammendment,-1,1,cd-3557,None,"[[""guns"", ""banned"", ""needed"", ""domestic"", ""iss...","[[""NNS"", ""VBN"", ""VBN"", ""JJ"", ""NN""], [""JJ"", ""NN...",guns banned needed domestic issue. second amme...,"[""second ammendment""]",second ammendment,0,1
1,27083,Guns should be banned because they are not nee...,british,None,british,-1,1,cd-3557,None,"[[""guns"", ""banned"", ""needed"", ""domestic"", ""iss...","[[""NNS"", ""VBN"", ""VBN"", ""JJ"", ""NN""], [""JJ"", ""NN...",guns banned needed domestic issue. second amme...,"[""british""]",british,0,1
2,27083,Guns should be banned because they are not nee...,age america,None,age america,-1,1,cd-3557,None,"[[""guns"", ""banned"", ""needed"", ""domestic"", ""iss...","[[""NNS"", ""VBN"", ""VBN"", ""JJ"", ""NN""], [""JJ"", ""NN...",guns banned needed domestic issue. second amme...,"[""age america""]",age america,0,1
3,27083,Guns should be banned because they are not nee...,close enemies,None,close enemies,-1,1,cd-3557,None,"[[""guns"", ""banned"", ""needed"", ""domestic"", ""iss...","[[""NNS"", ""VBN"", ""VBN"", ""JJ"", ""NN""], [""JJ"", ""NN...",guns banned needed domestic issue. second amme...,"[""close enemies""]",close enemies,0,1
4,27083,Guns should be banned because they are not nee...,easier crimes,None,easier crimes,-1,1,cd-3557,None,"[[""guns"", ""banned"", ""needed"", ""domestic"", ""iss...","[[""NNS"", ""VBN"", ""VBN"", ""JJ"", ""NN""], [""JJ"", ""NN...",guns banned needed domestic issue. second amme...,"[""easier"", ""crimes""]",easier crimes,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8080,2827,Making firearms illegal only stops law abiding...,only difference,None,only difference,-1,1,cd-558915,None,"[[""making"", ""firearms"", ""illegal"", ""stops"", ""l...","[[""VBG"", ""NNS"", ""JJ"", ""VBZ"", ""NN"", ""VBG"", ""NNS...",making firearms illegal stops law abiding citi...,"[""only difference""]",only difference,0,1
8081,2827,Making firearms illegal only stops law abiding...,black market distribution profitsome drug users,None,black market distribution profitsome drug users,-1,1,cd-558915,None,"[[""making"", ""firearms"", ""illegal"", ""stops"", ""l...","[[""VBG"", ""NNS"", ""JJ"", ""VBZ"", ""NN"", ""VBG"", ""NNS...",making firearms illegal stops law abiding citi...,"[""black market"", ""distribution"", ""profitsome d...",black market distribution profitsome drug users,0,1
8082,2827,Making firearms illegal only stops law abiding...,criminals,None,criminals,-1,1,cd-558915,None,"[[""making"", ""firearms"", ""illegal"", ""stops"", ""l...","[[""VBG"", ""NNS"", ""JJ"", ""VBZ"", ""NN"", ""VBG"", ""NNS...",making firearms illegal stops law abiding citi...,"[""criminals""]",criminals,0,1
8083,2496,"Yeah, I thought that was hilarious.",hilarious,None,hilarious,-1,1,cd-558927,None,"[[""yeah"", ""thought"", ""hilarious""]]","[[""UH"", ""VBD"", ""JJ""]]",yeah thought hilarious.,"[""hilarious""]",hilarious,0,1
